In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
</td>
<td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/instances/create?download_url=https://raw.githubusercontent.com/GoogleCloudDataproc/cloud-dataproc/ai-ml-samples/interactive/ImageClassificationInSpark.ipynb">
<img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
</a>
</td>
<td style="text-align: center">
<a href="https://github.com/GoogleCloudDataproc/cloud-dataproc/ai-ml-samples/interactive/ImageClassificationInSpark.ipynb">
<img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
</a>
</td>
</table>

## Overview

In this tutorial, you build a pipeline for predicting customer churn using [Apache Spark](https://spark.apache.org/), [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html), and the [Hugging Face Transformers](https://huggingface.co/docs/transformers/en/index) library. 

This tutorial is intended to run on [Dataproc Serverless](https://cloud.google.com/dataproc-serverless/docs/guides/create-serverless-sessions-templates) on Google Cloud.

## Get started

1. Create a Dataproc-enabled [Vertex AI workbench instance](https://cloud.google.com/vertex-ai/docs/workbench/instances/create-dataproc-enabled) or use an existing instance.
2. Enable [Private Google Access](https://cloud.google.com/dataproc-serverless/docs/concepts/network#private-google-access-requirement) on a subnet in your project.
3. Setup [Public NAT](https://cloud.google.com/nat/docs/set-up-manage-network-address-translation#create-nat-gateway) to download models from Hugging Face. See [Dataproc Serverless network configuration](https://cloud.google.com/dataproc-serverless/docs/concepts/network#subnetwork_requirements).
4. Create a [serverless runtime template](https://cloud.google.com/dataproc-serverless/docs/quickstarts/jupyterlab-sessions#dataproc_create_serverless_runtime_template-JupyterLab) and connect to a [remote kernel](https://cloud.google.com/vertex-ai/docs/workbench/instances/create-dataproc-enabled#serverless-spark).

Uncomment and run this cell if not using Dataproc Serverless version 1.2 or 2.2+.

In [ ]:
# pip install xgboost `transformers[torch]`

Import libraries.

In [ ]:
from google.cloud import bigquery
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import when, col, regexp_replace, trim
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, FloatType, IntegerType, StringType
from pyspark.sql.functions import col, isnan, when, count, lit, udf

from transformers import pipeline
from xgboost.spark import SparkXGBClassifier

Set a name for an existing [Cloud Storage](https://cloud.google.com/storage/docs/creating-buckets) bucket. 

In [ ]:
BUCKET = "YOUR-BUCKET"

Set your project name.

In [ ]:
PROJECT_ID = "YOUR-PROJECT-ID"

# Data transformations and exploratory analysis

In this tutorial you use the [IBM Telco Customer Churn](https://github.com/IBM/telco-customer-churn-on-icp4d) dataset. This dataset contains customer demographic and account information for a fictional telecommunications company. You use it to predict which customers are at risk of churn.

In [ ]:
# Specify the path to the CSV file
file_path = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/refs/heads/master/data/Telco-Customer-Churn.csv"

# Read the CSV file into a Spark DataFrame. Use pd.read_csv to read directly
# from an https link and then convert to Spark.
df = spark.createDataFrame(
    pd.read_csv(file_path)
)

Check for null values in the data, specifically in the `MonthlyCharges` and `TotalCharges` columns.

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ["MonthlyCharges", "TotalCharges"]]).show()

Check if `MonthlyCharges` and `TotalCharges` have any misformatted values.

In [ ]:
df.select([count(when(col(c).cast("float").isNull(), c)).alias(c) for c in ["MonthlyCharges", "TotalCharges"]]).show()

Drop null values and cast remaining values to float in TotalCharges to ensure data consistency required for machine learning model training. String and null values can cause errors.

In [ ]:
# Drop records where TotalCharges has null or string values that can't be cast to float
df = df.filter(col("TotalCharges").cast("float").isNotNull())

# Transform the data type of total_charges to float
df = df.withColumn("TotalCharges", col("TotalCharges").cast(FloatType()))

df.show(10)

## Visualize the data

Visualize churn rates by account types.

In [ ]:
# Aggregate data
df_plot_data = df.groupBy("Contract", "Churn").count()

# Convert to Pandas DataFrame for plotting
pdf_plot_data = df_plot_data.toPandas()

# Pivot DataFrame to get "contract" as index, "churn" as columns, and "count" as values
pdf_pivoted = pdf_plot_data.pivot(index='Contract', columns='Churn', values='count')

# Plot
pdf_pivoted.plot(kind='bar', stacked=True, figsize=(10, 7), title="Customer Count by Contract Type and Churn")

Next, visualize churn rate by account tenure.

In [ ]:
# Aggregate data
histogram_data = df.groupBy("Tenure", "Churn").count().orderBy("Tenure")

# Convert to Pandas DataFrame
pandas_histogram_data = histogram_data.toPandas()

# Pivot the data for stacked bar chart
pandas_histogram_data_pivot = pandas_histogram_data.pivot(index='Tenure', columns='Churn', values='count').fillna(0)

# Plot
pandas_histogram_data_pivot.plot(
    kind='bar', 
    stacked=True, 
    figsize=(12, 7),  # Adjust figure size as needed
    title="Account Tenure by Churn",
    xlabel="Tenure (Months)",
    ylabel="Count"
)

## Structured data transformations

Convert the `Tenure` field from a monthly bucket to a yearly bucket. This transforms a continuous variable into categorical groups, which can help a model identify non-linear relationships with churn and reduce the impact of outliers in the data.

In [ ]:
# Bucket the "Tenure" field due to disribution
df = df.withColumn("TenureBucket", 
    when(col("Tenure") <= 12, "Tenure_0-12") \
    .when((col("Tenure") > 12) & (col("Tenure") <= 24), "Tenure_12-24") \
    .when((col("Tenure") > 24) & (col("Tenure") <= 48), "Tenure_24-48") \
    .when((col("Tenure") > 48) & (col("Tenure") <= 60), "Tenure_48-60") \
    .otherwise("Tenure_GT_60"))

# Drop the original "Tenure" column
df = df.drop("Tenure")

Convert boolean columns to integer values.

In [ ]:
# Transform boolean fields to 1 and 0
for column in ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']:
    df = df.withColumn(column, when(col(column) == True, 1).otherwise(0))

# Change "Churn" values to 1 and 0
df = df.withColumn("Churn", when(col("Churn") == True, 1).otherwise(0)).cache()

Shorten categorical strings to make them easier for the model to process.

In [ ]:
# Replace "No internet service" with "No" in specified columns
for column in ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']:
    df = df.withColumn(column, when(col(column) == "No internet service", "No").otherwise(col(column)))

## Unstructured data transformations

Use the Hugging Face `transformers` library to perform sentiment analysis on a series of chat transcripts. Hugging Face provides a library of open source models to select from for a wide variety of use cases.

This tutorial uses [DistilBert model](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english). This model performs reasonably well while performing inference quickly. There are other models to choose from that have tradeoffs on performance, speed, accuracy, and model size.

This workload will take about 9 minutes to complete.

In [ ]:
# Specify the model name and revision - downloads from Huggingface
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b" 

# Load pre-trained sentiment analysis model with specified model and revision
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, revision=model_revision)

# Define a UDF to perform sentiment analysis
@udf(returnType=StringType())
def analyze_sentiment(text):
    try:
        result = sentiment_pipeline(text)[0]
        return result['label']
    except Exception as e:
        print(f"Error processing text: {text}. Error: {e}")
        return None

# Specify the path to the JSON file in GCS
file_path = "gs://data-analytics-demos/churn/chat_transcripts.json"

# Read the JSON file into a Spark DataFrame
df_sentiment = spark.read.json(file_path)

# Rename the "support_transcript" column to "text" (required for this particular model)
df_sentiment = df_sentiment.withColumnRenamed("support_transcript", "text")

# Remove records with null values for text
df_sentiment = df_sentiment.filter(df_sentiment["text"].isNotNull() & (df_sentiment["text"] != ""))

# Analyze the "support_transcript" field for sentiment and retain "customer_id"
df_sentiment = df_sentiment.withColumn("sentiment", analyze_sentiment(df_sentiment["text"])).cache()

# Display the results
df_sentiment.show()

## Join data

Combine the transformed transcript data to the customer churn data, then save this to Cloud Storage.

In [ ]:
# Join the transformed and sentiment data, and save to GCS

# Specify the output path in GCS
training_data_path = f"gs://{BUCKET}/curated/data/training_data"

# Join the DataFrames on customer_id
df_combined = df_sentiment.join(df, on="CustomerID", how="inner") 

# # Write the DataFrame to GCS in Parquet format
df_combined.write.format("parquet").mode("overwrite").save(training_data_path)

# Model training and evaluation

Next, you train an XGBoost model using the newly processed data. 

XGBoost does not currently support Spark's [dynamic resource allocation](https://spark.apache.org/docs/3.5.1/job-scheduling.html#dynamic-resource-allocation) feature, a feature of Apache Spark to dynamically allocate additional Spark executors to the environment. As Dataproc Serverless relies on dynamic resource allocation for autoscaling, this means [XGBoost does not autoscale on Dataproc Serverless](https://cloud.google.com/dataproc-serverless/docs/concepts/versions/spark-runtime-2.2#xgboost_libraries). The number of executors can be manually configured using the Spark configuration property `spark.executor.instances`.

To use XGBoost, disable dynamic allocation first by restarting the Spark Session.

In [ ]:
spark.stop()
spark = SparkSession.builder.config("spark.dynamicAllocation.enabled", "false").getOrCreate()

Load the data again in the new Spark Session.

In [ ]:
df_train = spark.read.parquet(training_data_path)

## Perform feature engineering

Convert the data to be usable with SparkML and SparkXGBoost.
- String Indexers map the string samples in the data to numbers, which "shrinks" the size of the data being processed.
- One Hot Encoders convert the categorical data into sparse vectors.
- A Vector Assembler is used to format the data per SparkML's requirements.

In [ ]:
categorical_cols = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                   'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 
                   'PaperlessBilling', 'PaymentMethod', 'TenureBucket', 'sentiment']
numerical_cols = ['SeniorCitizen', 'MonthlyCharges', 'TotalCharges'] 

indexers = [StringIndexer(inputCol=c, outputCol=c + "_index", handleInvalid="keep") for c in categorical_cols]  
encoders = [OneHotEncoder(inputCol=c + "_index", outputCol=c + "_encoded", handleInvalid="keep") for c in categorical_cols] 

assembler = VectorAssembler(inputCols=[c + "_encoded" for c in categorical_cols] + numerical_cols, outputCol="features", handleInvalid="keep") 

# Apply transformations:
indexed_df = Pipeline(stages=indexers).fit(df_train).transform(df_train)
encoded_df = Pipeline(stages=encoders).fit(indexed_df).transform(indexed_df)
transformed_data = assembler.transform(encoded_df)

## Train the model and generate predictions

Use the [SparkXGBoostClassifier](https://xgboost.readthedocs.io/en/latest/tutorials/spark_estimator.html#sparkxgbclassifier) to create a SparkML model. This is passed into a SparkML [pipeline](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.Pipeline.html) object.

In [ ]:
xgb = SparkXGBClassifier(label_col="Churn", features_col="features", num_workers=2, missing=0.0)

pipeline = Pipeline(stages=[xgb])

Split the data into training and test datasets.

In [ ]:
train_data, test_data = transformed_data.randomSplit([0.8, 0.2], seed=42) 

Fit the model using the training data.

In [ ]:
model = pipeline.fit(train_data)
print("Model training successful.")

Generate predictions on the test data.

In [ ]:
predictions = model.transform(test_data)

## Evaluate the model

Evaluate the performance of the model. These metrics provide a means to determine how effective a model is at predicting outcomes on new data. It is useful to track these metrics overtime. The metrics used are:
- [AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)
- [Accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy-precision-recall#accuracy)
- [F1 score](https://developers.google.com/machine-learning/crash-course/classification/accuracy-precision-recall#expandable-2)
- [Precision](https://developers.google.com/machine-learning/crash-course/classification/accuracy-precision-recall#precision)
- [Recall](https://developers.google.com/machine-learning/crash-course/classification/accuracy-precision-recall#recall_or_true_positive_rate)

In [ ]:
evaluator_binary = BinaryClassificationEvaluator(labelCol="Churn", rawPredictionCol="rawPrediction")
auc = evaluator_binary.evaluate(predictions, {evaluator_binary.metricName: "areaUnderROC"})
print(f"Area Under ROC (AUC): {auc:.4f}")

evaluator_multi = MulticlassClassificationEvaluator(labelCol="Churn", predictionCol="prediction")
accuracy = evaluator_multi.evaluate(predictions, {evaluator_multi.metricName: "accuracy"})
print(f"Accuracy: {accuracy:.4f}")

f1 = evaluator_multi.evaluate(predictions, {evaluator_multi.metricName: "f1"})
print(f"F1 Score: {f1:.4f}")

weighted_precision = evaluator_multi.evaluate(predictions, {evaluator_multi.metricName: "weightedPrecision"})
print(f"Weighted Precision: {weighted_precision:.4f}")

weighted_recall = evaluator_multi.evaluate(predictions, {evaluator_multi.metricName: "weightedRecall"})
print(f"Weighted Recall: {weighted_recall:.4f}")

## Save to Cloud Storage

Save the train data, test data, and XGBoost model to Cloud storage.

In [ ]:
# Specify GCS bucket training data path
data_path = f"gs://{BUCKET}/curated/models/churn_prediction"

# Save transformed training data
train_data.write.parquet(f"{data_path}/train_data", mode="overwrite")

# Save transformed test data
test_data.write.parquet(f"{data_path}/test_data", mode="overwrite")

# Save the trained model to GCS:
model.write().overwrite().save(f"{data_path}/xgboost_model")

# Run batch inference on Spark

Load the model and test data from Cloud Storage into Spark.

In [ ]:
# Specify my GCP bucket
model_bucket = f"gs://{BUCKET}/curated/models/churn_prediction"

# Load transformed training data
test_data_loaded = spark.read.parquet(f"{model_bucket}/test_data")

# Load the trained model
model_loaded = PipelineModel.load(f"{model_bucket}/xgboost_model")

Generate predictions on the test data.

In [ ]:
predictions = model_loaded.transform(test_data_loaded)

# Filter down to only include the customer_id and prediction columns
predictions = predictions.select("customerID", "prediction", "rawPrediction", "probability")

# Display sample data
predictions.show()

### Save predictions to BigQuery

Save the data to BigQuery. While out of scope for this tutorial, saving to BigQuery enables additional use cases such as joining this data with other BigQuery data or creating visualizations using a tool such as [Looker](cloud.google.com/looker).

Create a BigQuery dataset to store your predictions in.

In [ ]:
# Construct a BigQuery client object.
bigquery_client = bigquery.Client()

# Set dataset_id to the ID of the dataset to create.
dataset_id = f"{PROJECT_ID}.predictions"

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "us-central1"

# Send the dataset to the API for creation, with an explicit timeout.
dataset = bigquery_client.create_dataset(dataset, timeout=30)  # Make an API request.

Save the predictions directly from Spark to BigQuery.

In [ ]:
# Define table name to save predictions
table_name = 'predictions'

# Write predictions DataFrame to table
predictions.write \
    .format("bigquery") \
    .mode("overwrite") \
    .option("writeMethod", "direct") \
    .save(f'{dataset_id}.{table_name}')

# Clean up

[Delete](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) your project or use the following steps to delete project resources.

Uncomment to delete the BigQuery dataset and table.

In [ ]:
# bigquery_client.delete_dataset(dataset_id, delete_contents=True)

Uncomment to delete either the Cloud Storage bucket or only the content this tutorial created.

In [ ]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET)

# Uncomment to DELETE ONLY THE CONTENTS created in this tutorial.
# for blob in bucket.list_blobs(prefix="curated"):
#     blob.delete()

# Uncomment to DELETE THE BUCKET.
# bucket.delete()

To delete the Vertex AI Workbench instance, go to the [Instances](https://console.cloud.google.com/vertex-ai/workbench/instances) page, select your instance and click **DELETE**.

To delete the Dataproc Serverless interactive session, go to the [Interactive Sessions](https://console.cloud.google.com/dataproc/interactive) page, click your session and click **DELETE**.

# Next steps

- [Try more tutorials using Spark and Dataproc Serverless for machine learning](https://github.com/GoogleCloudDataproc/cloud-dataproc/tree/master/notebooks/ai-ml)
- [Learn more about Dataproc Serverless](https://cloud.google.com/dataproc-serverless/docs)
- [Learn more about Vertex AI](https://cloud.google.com/vertex-ai)